In [84]:
%config Completer.use_jedi = False

In [88]:
locations_competition_df = dataiku.Dataset("locations_competition").get_dataframe()
locations_competition_df = locations_competition_df[~locations_competition_df["distance_between_locations"].isnull()]
locations_competition_df = locations_competition_df.reset_index()
locations_competition_df["location_identifier"][0]
locations_competition_df["in_isochrone_location_identifier"][0]

In [96]:
locations_competition_df

index                                location_identifier    isochrone_type                   in_isochrone_location_identifier  distance_between_locations
0        0          1 Route De Francheleins - 01090 Montceaux  isochrone_30_min                13-15 Rue De La Barre - 71000 Mâcon                   24.078197
1        1          1 Route De Francheleins - 01090 Montceaux  isochrone_30_min   339 Rue Nationale - 69400 Villefranche-sur-Saône                   13.071563
2        2          1 Route De Francheleins - 01090 Montceaux  isochrone_30_min  420 Rue Philippe Héron - 69400 Villefranche-su...                   13.643218
3        3        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_30_min             Rue Des Chézeaux - 21560 Arc-sur-Tille                   14.031227
4        4        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_30_min                      Rue De La Gare - 21110 Genlis                   19.616362
5        5        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_30_min           Rue Anatole France, - 21120 Is-sur-Tille                   24.290996
6        6        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_30_min  Place De La Liberté - 21800 Chevigny-Saint-Sau...                    8.443926
7        7        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_30_min                8 Rue Paul Langevin - 21300 Chenôve                    5.129946
8        8        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_30_min       51 Rue René Cassin - 21850 Saint-Apollinaire                    3.755413
9        9        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_30_min                    49 Impasse Arago - 21110 Genlis                   21.225417
10      10        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_30_min    25 Rue François Mitterrand - 21120 Is-sur-Tille                   25.399015
11      11        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_30_min                      2 Rue Bannelier - 21000 Dijon                    2.700636
12      12        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_30_min            2 Avenue Raymond Poincaré - 21000 Dijon                    2.326338
13      13        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_30_min               Rue Du Faubourg Raines - 21000 Dijon                    3.172233
14      14        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_30_min                  129 Route De Dijon - 21200 Beaune                   38.423487
15      15        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_30_min      1 Rue Des Tilleuls - 21910 Noiron-sous-Gevrey                   13.055848
16      16        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_15_min               Rue Du Faubourg Raines - 21000 Dijon                    3.172233
17      17        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_15_min  Place De La Liberté - 21800 Chevigny-Saint-Sau...                    8.443926
18      18        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_15_min                8 Rue Paul Langevin - 21300 Chenôve                    5.129946
19      19        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_15_min       51 Rue René Cassin - 21850 Saint-Apollinaire                    3.755413
20      20        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_15_min                      2 Rue Bannelier - 21000 Dijon                    2.700636
21      21        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_15_min            2 Avenue Raymond Poincaré - 21000 Dijon                    2.326338
22      22        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_15_min     1A Rue Marie Antoinette Tonnelat - 21000 Dijon                    3.196652
23      23        1 Rue Albert Et André Claudot - 21000 Dijon  isochrone_30_min     1A Rue Marie Antoinette Tonnelat - 21000 Dijon                    3.196652
24      24      1 Rue Des Till

In [97]:
import dataiku
import pandas as pd
import numpy as np
import json
from ast import literal_eval
import flask
from flask import request
from geomatic.geographic_handling import read_geo_point, compute_geo_points_collection_center
import time  

start_time = time.time()
# Loading the locations_competition dataset :
locations_competition_df = dataiku.Dataset("locations_competition").get_dataframe()
locations_competition_df = locations_competition_df[~locations_competition_df["distance_between_locations"].isnull()]
locations_competition_df = locations_competition_df.reset_index()
location = locations_competition_df["location_identifier"][0]
location_2 = locations_competition_df["in_isochrone_location_identifier"][0]
print("location 1 : {} | location 2 : {}".format(location, location_2))

# Variables definition : 
project_key = dataiku.get_custom_variables()["projectKey"]
client = dataiku.api_client()
project = client.get_project(project_key)
variables = project.get_variables()["standard"]
isochrones_range_min = variables["isochrones_range_min"]
isochrones_labels = ["isochrone_{}_min".format(isochrone) \
                     for isochrone in isochrones_range_min]
print("BACKEND_LOG_MESSAGE : isochrones_labels : {}".format(isochrones_labels))
analyze_pois = variables["analyze_pois"]
analyze_customers = variables["analyze_customers"]

# Loading locations data :
locations_isochrones_df = dataiku.Dataset("locations_isochrones").get_dataframe()
dataframe_columns = locations_isochrones_df.columns
application_filtering_columns = [col for col in dataframe_columns if \
                                 ((col not in isochrones_labels) and \
                                  (col not in ["latitude", "longitude", "geo_point"]) and \
                                  (col!="location_identifier")
                                 )]
print("BACKEND_LOG_MESSAGE : application_columns : {}".format(application_filtering_columns))
#columns_unique_values = {col:list(np.unique(locations_isochrones_df[col]))\
#                                     for col in dataframe_columns}

locations_identifiers = list(locations_isochrones_df["location_identifier"])
locations_geo_points = list(locations_isochrones_df["geo_point"])
locations_geo_points = {location:read_geo_point(geo_point) \
                        for location, geo_point in zip(locations_identifiers, locations_geo_points)}
locations_data = {location:{} for location in locations_identifiers}

# Loading locations POI's data :
if analyze_pois:
    points_of_interest_tags = variables["points_of_interest_tags"]
    points_of_interest_enrichments = variables["points_of_interest_enrichments"]
    points_of_interest_all_tags = points_of_interest_tags+points_of_interest_enrichments
    try:
        locations_pois_df = dataiku.Dataset("locations_points_of_interest").get_dataframe()
        unique_pois_df = dataiku.Dataset("unique_points_of_interest").get_dataframe()
        points_of_interest_computed = True
    except:
        points_of_interest_computed = False
        
    if points_of_interest_computed:
        pois_ids = list(unique_pois_df["poi_id"])
        poi_attributes = {poi_id:{} for poi_id in pois_ids}

        for poi_data in unique_pois_df.itertuples():
            poi_id = poi_data.poi_id
            poi_attributes[poi_id]["coordinates"] = [poi_data.poi_lon, poi_data.poi_lat]
            poi_attributes[poi_id]["poi_all_information"] = poi_data.tags
            poi_attributes[poi_id]["poi_tag_types"] = []
            poi_attributes[poi_id]["poi_tags"] = []

            for tag_type in points_of_interest_all_tags:
                poi_has_tag_type = not pd.isnull(getattr(poi_data, tag_type))
                if poi_has_tag_type:
                    poi_attributes[poi_id]["poi_tag_types"].append(tag_type)
                    poi_attributes[poi_id]["poi_tags"].append(getattr(poi_data, tag_type))

# Loading locations customers data :
if analyze_customers:
    try:
        customers_df = dataiku.Dataset("customers_prepared").get_dataframe()
        locations_customers_df = dataiku.Dataset("locations_customers").get_dataframe()
        customers_computed = True
    except:
        customers_computed = False
    
    if customers_computed:
        customers_geo_points = {customer:read_geo_point(geo_point) \
                                for customer, geo_point \
                                in zip(customers_df["customer_id"], customers_df["geo_point"])}
    
    
    
                
# Computing backend data :
for isochrone_type in isochrones_labels:
    focus_isochrones = list(locations_isochrones_df[isochrone_type])    
    
    for location, isochrone_geojson in zip(locations_identifiers, focus_isochrones):
        locations_data[location][isochrone_type] = {}
        locations_data[location][isochrone_type]["isochrone_geojson"] = literal_eval(isochrone_geojson)
        
        # POI's :
        if analyze_pois and points_of_interest_computed:
            sampled_locations_pois_df = locations_pois_df[(locations_pois_df["isochrone_type"]==isochrone_type) & (locations_pois_df["poi_belonging_location"]==location)]
            location_isochrone_pois = list(sampled_locations_pois_df["poi_belonging_location"])
            locations_data[location][isochrone_type]["pois_list"] = location_isochrone_pois
        
        # Customers : 
        if analyze_customers and customers_computed:
            sampled_locations_customers_df = locations_customers_df[(locations_customers_df["isochrone_type"]==isochrone_type) & (locations_customers_df["location_identifier"]==location)]
            location_isochrone_customers = list(sampled_locations_customers_df["included_customer_id"])
            locations_data[location][isochrone_type]["customers_list"] = location_isochrone_customers

/data/dataiku/datadir/jupyter-run/dku-workdirs/DISTR_SPATIAL/scenario_tester89b26783-dataiku_user/project-python-libs/DISTR_SPATIAL/python/geomatic/geographic_handling.py:9: DeprecationWarning: invalid escape sequence \(
  geo_point = re.sub("POINT|\(|\)","", geo_point)
/data/dataiku/datadir/jupyter-run/dku-workdirs/DISTR_SPATIAL/scenario_tester89b26783-dataiku_user/project-python-libs/DISTR_SPATIAL/python/geomatic/geographic_handling.py:109: DeprecationWarning: invalid escape sequence \[
  overpass_parameters = re.sub("[\[\],]", "", overpass_parameters)


location 1 : 1 Route De Francheleins - 01090 Montceaux | location 2 : 13-15 Rue De La Barre - 71000 Mâcon
BACKEND_LOG_MESSAGE : isochrones_labels : ['isochrone_5_min', 'isochrone_15_min', 'isochrone_30_min']
BACKEND_LOG_MESSAGE : application_columns : ['shop', 'group_name', 'shop_type', 'shop_category', 'address', 'city', 'department', 'region', 'country_iso', 'isochrone_5_min_total_pop', 'isochrone_5_min_area', 'isochrone_5_min_reachfactor', 'isochrone_15_min_total_pop', 'isochrone_15_min_area', 'isochrone_15_min_reachfactor', 'isochrone_30_min_total_pop', 'isochrone_30_min_area', 'isochrone_30_min_reachfactor']


In [98]:
isochrone_type = "isochrone_30_min"
geojson = locations_data[location][isochrone_type]["isochrone_geojson"]
geojson_2 = locations_data[location_2][isochrone_type]["isochrone_geojson"]

In [100]:
location

'Rue Ledru Rollin - 58170 Luzy'

In [99]:
geojson

{'type': 'Feature',
 'properties': {'group_index': 2,
  'value': 1800.0,
  'center': [3.963556555625968, 46.79057943733429],
  'area': 1219.73591229,
  'reachfactor': 0.1553,
  'total_pop': 23107.0,
  'envelope': {'min_lat': 46.581528,
   'min_lon': 3.606318,
   'max_lat': 46.986104,
   'max_lon': 4.236581},
  'envelope_str': '(46.581528,3.606318,46.986104,4.236581)',
  'popupContent': 'Rue Ledru Rollin - 58170 Luzy , Isochrone 0 min, total_pop : 23107.0, area : 1219.73591229, reachfactor : 0.1553'},
 'geometry': {'coordinates': [[[3.606318, 46.845778],
    [3.609021, 46.845627],
    [3.611725, 46.845476],
    [3.615225, 46.845366],
    [3.617784, 46.845694],
    [3.624036, 46.846574],
    [3.633109, 46.846136],
    [3.636774, 46.845118],
    [3.643102, 46.84183],
    [3.645384, 46.840296],
    [3.644254, 46.829776],
    [3.643537, 46.829286],
    [3.641311, 46.820355],
    [3.638348, 46.817761],
    [3.635866, 46.816476],
    [3.634294, 46.815694],
    [3.631624, 46.814408],
    [3.63

In [82]:
import dataiku

variables = dataiku.api_client()\
.get_project(dataiku.get_custom_variables()["projectKey"])\
.get_variables()

isochrones_range_min = variables["standard"].get("isochrones_range_min")
isochrones_in_global_variables = (isochrones_range_min is not None)

def do():
    choices = []
    if isochrones_in_global_variables:
        for isochrone_type in isochrones_range_min:
            isochrone_column_for_pois = "isochrone_{}_min".format(isochrone_type)
            choices.append({"value": isochrone_column_for_pois,
                            "label": "Look for POIs in column '{}'".format(isochrone_column_for_pois)})
            
    else:
        if len(isochrones_range_min)>0:
            message = "ERROR : Variable 'isochrones_range_min' is empty. Rebuild isochrones ('Isochrones' section)"
        else:
            message = "ERROR : Variable 'isochrones_range_min' is not defined in the global variables. Rebuild isochrones ('Isochrones' section)"
        choices.append({"value": "isochrones_dataset_not_yet_built",
                        "label": message})
    
    return {"choices": choices}
        

In [83]:
do()

{'choices': [{'value': 'isochrone_5_min',
   'label': "Look for POIs in column 'isochrone_5_min'"},
  {'value': 'isochrone_15_min',
   'label': "Look for POIs in column 'isochrone_15_min'"},
  {'value': 'isochrone_30_min',
   'label': "Look for POIs in column 'isochrone_30_min'"}]}

In [60]:
import dataiku
project = dataiku.api_client().get_project(dataiku.get_custom_variables()["projectKey"])

variables = project.get_variables()
variables

{'standard': {'distribution_locations_type': 'addresses',
  'analyze_customers': False,
  'customer_locations_type': 'addresses',
  'isochrones_api_to_use': 'open_route_service',
  'isochrones_range_min': [5, 15, 30],
  'transportation_mode_open_route_service': 'driving-car',
  'isochrones_attributes_open_route_service': ['total_pop',
   'area',
   'reachfactor'],
  'analyze_pois': True,
  'points_of_interest_tags': ['shop', 'leisure', 'historic', 'tourism'],
  'enrich_pois': True,
  'points_of_interest_enrichments': ['brand', 'operator'],
  'identifier_columns': ['address'],
  'api_key': '5b3ce3597851110001cf6248224e0c0c65b941c4adc0e5494229252e',
  'open_route_service': {'transportation_mode': 'driving-car',
   'isochrones_attributes': ['total_pop', 'area', 'reachfactor']}},
 'local': {'distribution_locations_type': 'addresses',
  'analyze_customers': False,
  'customer_locations_type': 'addresses',
  'isochrones_api_to_use': 'open_route_service',
  'isochrones_range_min': ['5', '15',

In [61]:
isochrones_columns = ["isochrone_{}_min".format(minute_value) for minute_value in [5, 10, 15, 30, 45, 60]]
dataset = project.get_dataset("locations_isochrones")
dataset_schema = dataset.get_schema()["columns"]
isochrones_column_in_dataset = []
for column_params in dataset_schema:
    if column_params["name"] in isochrones_columns:
        isochrones_column_in_dataset.append(column_params["name"])

In [66]:
dataset.run_checks?

In [67]:
dataset.get_metric_history("python:records_in_dataset:Python probe")["values"][-1]["value"]

73.0

In [22]:
dataset.get_metric_history?

In [59]:
dataset.

{'checklists': {'checklists': []}, 'tags': [], 'custom': {'kv': {}}}

In [45]:
def get_dataset_metric_information(project, dataset_name, metric):
    """
    'metric' in ['basic:COUNT_COLUMNS', 'basic:COUNT_FILES', 'basic:SIZE', 'records:COUNT_RECORDS']
    """
    dataset = project.get_dataset(dataset_name)
    metric_information = dataset.get_metric_history(metric)

    return metric_information

In [57]:
def get_dataset_metric_information(project, dataset_name, metric):
    """
    'metric' in ['basic:COUNT_COLUMNS', 'basic:COUNT_FILES', 'basic:SIZE', 'records:COUNT_RECORDS']
    """
    from urllib.error import HTTPError
    try:
        dataset = project.get_dataset(dataset_name)
        if dataset.exists():
            metric_information = dataset.get_metric_history(metric)
        else:
            metric_information = "Dataset '{}' does not exist.".format(dataset_name)
    except HTTPError:
        metric_information = "Metric '{}' don't seems to be defined on this dataset. "\
        "Check the exact metric name with a metrics dataset "\
        "(See also : https://doc.dataiku.com/dss/latest/connecting/metrics.html).".format(metric)
    return metric_information

In [58]:
get_dataset_metric_information(project, "locations_isochrones", "records:COUNT_RECORDS")

DataikuException: com.dataiku.dip.server.controllers.NotFoundException: Metric: 'records:COUNT_RECORD_S' does not exist on dataset DISTR_SPATIAL.locations_isochrones.

In [0]:
dataiku.core.base.os.

In [85]:
"isochrone_15_min".split("_")[1]

'15'

In [86]:
len([])

0

In [0]:
def get_current_project_and_variables():
    project_key = dataiku.get_custom_variables()["projectKey"]
    client = dataiku.api_client()
    project = client.get_project(project_key)
    variables = project.get_variables()
    return project, variables

In [26]:
from dku_utils import (get_current_project_and_variables, update_recipe_ouput_schema)

In [27]:
RECIPE_FOR_LOCATIONS_POINTS_OF_INTEREST= "compute_locations_points_of_interest_agg"

In [66]:
project, variables = get_current_project_and_variables()
points_of_interest_tags = variables["standard"]["points_of_interest_tags"]
columns_tags_to_aggregate = ["has_{}".format(tag) for tag in points_of_interest_tags]

In [55]:
#points_of_interest_tags = ["has_shop", "has_leisure"]

In [56]:
recipe_for_locations_points_of_interest = project.get_recipe(RECIPE_FOR_LOCATIONS_POINTS_OF_INTEREST)

In [57]:
recipe_settings = recipe_for_locations_points_of_interest.get_settings()

In [67]:
recipe_new_aggregations = []
recipe_payload = recipe_settings.get_json_payload()
recipe_aggregations = recipe_payload["values"]
for aggregation in recipe_aggregations:
    aggregated_col = aggregation["column"]
    if aggregated_col in columns_tags_to_aggregate:
        aggregation["sum"] = True
        aggregation["$selected"] = True
    else: 
        aggregation["sum"] = False
        aggregation["$selected"] = False
    recipe_new_aggregations.append(aggregation)
recipe_payload["values"] = recipe_new_aggregations

In [68]:
recipe_settings.set_json_payload(recipe_payload)

In [69]:
recipe_settings.save()

{'msg': 'Updated recipe DISTR_SPATIAL.compute_locations_points_of_interest_agg'}

In [20]:
update_recipe_ouput_schema(project, RECIPE_FOR_LOCATIONS_POINTS_OF_INTEREST)

In [40]:
for old, new in zip(recipe_aggregations, recipe_new_aggregations):
    print("{} | {}".format(old["sum"], new["sum"]))
    #if old["sum"] != new["sum"]:
        #print(old["column"])

False | False
False | False
False | False
False | False
False | False
False | False
False | False
False | False
True | True
False | False
True | True
False | False
False | False
False | False
False | False
False | False
False | False
False | False
False | False
False | False
False | False
False | False
False | False
False | False
False | False
False | False
False | False
False | False


In [2]:
import dataiku
import re
from geomatic.utils import STARTER_PROJECT_VARIABLES

In [3]:
from dku_utils import switch_recipe_input,get_current_project_and_variables,\
get_flow_zone_id, move_dataset_in_flow_zone

In [4]:
project_key = dataiku.get_custom_variables()["projectKey"]
client = dataiku.api_client()
project = client.get_project(project_key)
flow = project.get_flow()
flow_zones = flow.list_zones()

In [5]:
flow_zones

[<dataikuapi.dss.flow.DSSFlowZone (id=1aYb4O1, name=points_of_interest_zone)>,
 <dataikuapi.dss.flow.DSSFlowZone (id=DezozOK, name=customers_zone)>,
 <dataikuapi.dss.flow.DSSFlowZone (id=E5kZMPY, name=inputs_zone)>,
 <dataikuapi.dss.flow.DSSFlowZone (id=default, name=Default)>,
 <dataikuapi.dss.flow.DSSFlowZone (id=hB8R0Xc, name=preprocessing_zone)>,
 <dataikuapi.dss.flow.DSSFlowZone (id=oek11ZE, name=isochrones_zone)>]

In [6]:
get_flow_zone_id('toto')

In [12]:
STARTER_PROJECT_VARIABLES["open_route_service"]

[]

In [13]:
project_key = dataiku.get_custom_variables()["projectKey"]
client = dataiku.api_client()
project = client.get_project(project_key)
variables = project.get_variables()
local_variables  = variables["local"]
for variable_name in STARTER_PROJECT_VARIABLES.keys():
    local_variables[variable_name] = STARTER_PROJECT_VARIABLES[variable_name]
isochrones_api_to_use = local_variables["isochrones_api_to_use"]
isochrones_api_suffix = "_{}".format(isochrones_api_to_use)

In [14]:
local_variables

{'open_route_service': [],
 'isochrones_range_min': [5, 15, 30],
 'points_of_interest_tags': ['shop',
  'leisure',
  'sport',
  'tourism',
  'historic',
  'amenity',
  'railway',
  'power',
  'line_management'],
 'isochrones_api_to_use': 'open_route_service',
 'locations_type': 'addresses',
 'identifier_columns': ['shop', 'address'],
 'api_key': '5b3ce3597851110001cf6248224e0c0c65b941c4adc0e5494229252e',
 'points_of_interest_enrichments': ['brand', 'operator', 'address']}

In [15]:
def loadProjectParameters(local_variables):   
    global_variables = {}
    
    for variable_name in local_variables.keys():
        if isochrones_api_suffix in variable_name:
            isochrone_api_variable_name = re.sub(isochrones_api_suffix, "", variable_name)
            variable_value = local_variables[variable_name]
            local_variables[isochrones_api_to_use].append({isochrone_api_variable_name:variable_value})
    
    return local_variables

In [16]:
test = loadProjectParameters(local_variables)

In [17]:
test["open_route_service"]

[]

In [18]:
test

{'open_route_service': [],
 'isochrones_range_min': [5, 15, 30],
 'points_of_interest_tags': ['shop',
  'leisure',
  'sport',
  'tourism',
  'historic',
  'amenity',
  'railway',
  'power',
  'line_management'],
 'isochrones_api_to_use': 'open_route_service',
 'locations_type': 'addresses',
 'identifier_columns': ['shop', 'address'],
 'api_key': '5b3ce3597851110001cf6248224e0c0c65b941c4adc0e5494229252e',
 'points_of_interest_enrichments': ['brand', 'operator', 'address']}

In [30]:
AVAILABLE_ISOCHRONES_APIS = ["open_route_service"]
STARTER_PROJECT_VARIABLES = {isochrone_api:list() for isochrone_api in AVAILABLE_ISOCHRONES_APIS}

In [31]:
STARTER_PROJECT_VARIABLES

{'open_route_service': []}

In [0]:
local_variables = loadProjectParameters(local_variables)
variables["standard"] = variables["local"]

In [0]:

    
project.set_variables(variables)

In [8]:
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
from geomatic.isochrones import isochronesManager
from geomatic.geographic_handling import compute_polygon_envelope
from geomatic.utils import from_minutes_to_seconds, from_seconds_to_minutes
from dku_utils import get_current_project_and_variables

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
project, variables = get_current_project_and_variables()
global_variables = variables["standard"]

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
api_key = global_variables["api_key"]
isochrones_api_to_use = global_variables["isochrones_api_to_use"]
isochrones_api_parameters = global_variables[isochrones_api_to_use]
isochrones_range_min = global_variables["isochrones_range_min"]
points_of_interest_tags = global_variables["points_of_interest_tags"]

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# Read recipe inputs
INPUT_DATASET_NAME = "locations_prepared" # locations_prepared_sampled / locations_prepared
locations_prepared = dataiku.Dataset(INPUT_DATASET_NAME)
locations_prepared_df = locations_prepared.get_dataframe()

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
locations_identifiers = list(locations_prepared_df["location_identifier"])

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: MARKDOWN
# # Isochrones computation

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
isochrones_manager = isochronesManager(isochrones_api_to_use, api_key, isochrones_api_parameters, isochrones_range_min)

In [10]:
isochrones_manager.client.isochrones?

In [6]:
import dataiku
customers_ds = dataiku.Dataset("locations_customers")
locations_ds = dataiku.Dataset("locations_isochrones")

In [7]:
customers_df = customers_ds.get_dataframe()

In [5]:
list(customers_df.columns)

['location_identifier',
 'location_id',
 'isochrone_type',
 'included_customer_id',
 'distance_customer_location',
 'isochrone_id',
 'isochrone_amplitude',
 'isochrone_label',
 'isochrone_data',
 'customer_id_denormalized',
 'latitude',
 'longitude',
 'geo_point',
 'city',
 'department',
 'region',
 'country_iso']

In [8]:
locations_df = locations_ds.get_dataframe()

In [13]:
set(locations_df.columns).difference(set(LOCATIONS_NO_FILTERING_COLUMNS))

{'city',
 'country_iso',
 'department',
 'group_name',
 'location_identifier',
 'region',
 'shop',
 'shop_category',
 'shop_type'}

In [12]:
LOCATIONS_NO_FILTERING_COLUMNS = [
    'address',
    'latitude',
    'longitude',
    'geo_point',
    'isochrone_5_min',
    'isochrone_5_min_total_pop',
    'isochrone_5_min_area',
    'isochrone_5_min_reachfactor',
    'isochrone_15_min',
    'isochrone_15_min_total_pop',
    'isochrone_15_min_area',
    'isochrone_15_min_reachfactor',
    'isochrone_30_min',
    'isochrone_30_min_total_pop',
    'isochrone_30_min_area',
    'isochrone_30_min_reachfactor',
    'isochrone_45_min',
    'isochrone_45_min_total_pop',
    'isochrone_45_min_area',
    'isochrone_45_min_reachfactor',
    'isochrone_60_min',
    'isochrone_60_min_total_pop',
    'isochrone_60_min_area',
    'isochrone_60_min_reachfactor',
    'isochrone_90_min',
    'isochrone_90_min_total_pop',
    'isochrone_90_min_area',
    'isochrone_90_min_reachfactor',
    'webapp_location_id',
    'location_id'
]

In [1]:
import dataiku

In [6]:
dataiku.api_client().get_default_project().get_variables()['standard']

{'distribution_locations_type': 'lat_lon',
 'analyze_customers': True,
 'customer_locations_type': 'lat_lon',
 'isochrones_api_to_use': 'open_route_service',
 'isochrones_range_min': [5, 15, 30],
 'transportation_mode_open_route_service': 'driving-car',
 'isochrones_attributes_open_route_service': ['total_pop',
  'area',
  'reachfactor'],
 'analyze_pois': True,
 'points_of_interest_tags': ['shop', 'leisure', 'historic', 'tourism'],
 'enrich_pois': True,
 'points_of_interest_enrichments': ['brand', 'operator'],
 'identifier_columns': ['address'],
 'api_key': '5b3ce3597851110001cf6248224e0c0c65b941c4adc0e5494229252e',
 'projectRandomKey': 'xxPFyGDh',
 'input_dataset': 'locations_dataset',
 'combine_customers_and_isochrones_data': True,
 'isochrones_for_customers': ['isochrone_5_min',
  'isochrone_15_min',
  'isochrone_30_min'],
 'isochrones_for_pois': ['isochrone_5_min',
  'isochrone_15_min',
  'isochrone_30_min'],
 'open_route_service': {'transportation_mode': 'driving-car',
  'isochron